# Russian Synodal Bible (1885)

## Import

### Libraries

In [1]:
import os
import re
import json
import time
import numpy as np
import pandas as pd 
from lxml import etree
import natasha
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, PER, NamesExtractor, Doc
#from bs4 import BeautifulSoup
#import requests

In [2]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

### Definitions

In [3]:
texts = './texts/fiction/'
bibleTXT = './texts/bible/sinodalnyi-perevod.txt'
libCols = ['author','pub_year','title','text']
tokenOHCO = ['title','part_num','para_num', 'sent_num', 'token_num']
bibleOHCO = ['test', 'book', 'chap', 'verse']
bibleJSON = './texts/bible/bible.json'
bibleXML = './texts/bible/bible.xml'
tokenCols = ['v_id', 'token_num', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']
chap_lines_re = '^===\s(\d{1,3})\s===$'
book_lines_re = '^==\s(.+)\s==$'

## Pre-Processing

### Import Text into DF

In [ ]:
with open(bibleTXT, 'r', encoding='windows-1251') as f: 
    bibleText = f.readlines()

bibliiaDf = pd.DataFrame(bibleText).rename(columns={0:'text'}).dropna()
bibliiaDf

### Find Parts

### Tidy Up

In [ ]:
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n', '')
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace('\]|\[|_|-|', '')
bibliiaDf = bibliiaDf.loc[bibliiaDf.text != '']
bibliiaDf

In [ ]:
chap_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(chap_lines_re)]
book_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(book_lines_re)]
test_line = bibliiaDf.loc[bibliiaDf.text.str.contains('От Матфея святое благовествование')]
stru_lines = chap_lines+book_lines

### Assign OHCO Labels

In [ ]:
bibliiaDf[bibleOHCO[0]] = np.where(bibliiaDf.index<test_line.index[0], 'O', 'N')
bibliiaDf[bibleOHCO[1]] = book_lines.text.str.extract(book_lines_re)
bibliiaDf[bibleOHCO[2]] = chap_lines.text.str.extract(chap_lines_re)
bibliiaDf[[bibleOHCO[3],'text']] = bibliiaDf.text.str.split(' ', 1, expand=True).iloc[:, [0, 1]]
bibliiaDf[bibleOHCO[1:3]] = bibliiaDf[['book','chap']].ffill()
bibliiaDf = bibliiaDf.drop(stru_lines.index, axis=0).reset_index()#.set_index(bibleOHCO)
bibliiaDf.index.name = 'v_id'
bibliiaDf = bibliiaDf[['test', 'book', 'chap', 'verse', 'text']]
bibliiaDf

### Assign Testament Label

### Index All Parts

### Assign Book Label

### Assign Chapter & Verse Labels

### Set Index & Remove Fluff

### Export to JSON

In [ ]:
bibliiaDf.to_json(bibleJSON, orient='index')

### Make XML

In [ ]:
root = etree.Element("bible")
print(root)
print(root.tag)
#bibleTree = etree.ElementTree()

In [ ]:
for v in BibDf.index.to_list(): 
    

In [ ]:
BibDf

In [ ]:
root.append(etree.Element("testament"))

In [ ]:
root.append(etree.Element(""))

In [ ]:
etree.tostring(root, pretty_print=True)

## Processing

In [6]:
BibDf = pd.read_json(bibleJSON, orient='index')
BibDf.index.name = 'v_id'
BibTextDf = BibDf[['text']]
BibLibDf = BibDf[bibleOHCO]
BibDf

,test,book,chap,verse,text
v_id,,,,,
0,O,Бытие,1,1,В начале сотворил Бог небо и землю.
1,O,Бытие,1,2,"Земля же была безвидна и пуста, и тьма над без..."
2,O,Бытие,1,3,И сказал Бог: да будет свет. И стал свет.
3,O,Бытие,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
4,O,Бытие,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
37105,N,Откровение святого Иоанна Богослова,22,17,И Дух и невеста говорят: прииди! И слышавший д...
37106,N,Откровение святого Иоанна Богослова,22,18,И я также свидетельствую всякому слышащему сло...
37107,N,Откровение святого Иоанна Богослова,22,19,и если кто отнимет что от слов книги пророчест...


In [11]:
#testsDict = dict(enumerate(BibDf.test.unique()))
testsDict = dict([(value, key) for key, value in dict(enumerate(BibDf.test.unique())).items()])
booksDict = dict([(value, key) for key, value in dict(enumerate(BibDf.book.unique())).items()])

In [ ]:
[x for x in BibDf.groupby(bibleOHCO[:1]).text]

In [ ]:
BibDf.loc[isinstance(BibDf.text, str)]

In [ ]:
[(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:1]).text if isinstance(x, str)]

In [8]:
TestsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:1]).text], columns=['test', 'text'])
TestsDf.index.name = 't_id'
TestsDf 

,test,text
t_id,,
0,N,"Родословие Иисуса Христа, Сына Давидова, Сына ..."
1,O,В начале сотворил Бог небо и землю. Земля же б...


In [12]:
BooksDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:2]).text], columns=[('test', 'book'), 'text'])
BooksDf[['test','book']] = pd.DataFrame(list(BooksDf[('test', 'book')]), index=BooksDf.index, columns=bibleOHCO[:2])
del BooksDf[('test', 'book')]
BooksDf = BooksDf.replace({"test": testsDict, "book": booksDict}).sort_values(by=bibleOHCO[:2], ascending=[True, True])
BooksDf = BooksDf.reset_index().drop(['index'], axis=1)
BooksDf.index.name = 'b_id'
BooksDf = BooksDf.replace({"test":dict(enumerate(BibDf.test.unique())), "book":dict(enumerate(BibDf.book.unique()))})
#BooksDf = BooksDf.reset_index().set_index(bibleOHCO[:2])
BooksDf 

,text,test,book
b_id,,,
0,В начале сотворил Бог небо и землю. Земля же б...,O,Бытие
1,"Вот имена сынов Израилевых, которые вошли в Е...",O,Исход
2,И воззвал Господь к Моисею и сказал ему из ски...,O,Левит
3,"И сказал Господь Моисею в пустыне Синайской, в...",O,Числа
4,"Сии суть слова, которые говорил Моисей всем Из...",O,Второзаконие
...,...,...,...
72,"Павел, волею Божиею Апостол Иисуса Христа, по ...",N,Второе послание к Тимофею святого апостола Павла
73,"Павел, раб Божий, Апостол же Иисуса Христа, по...",N,Послание к Титу святого апостола Павла
74,"Павел, узник Иисуса Христа, и Тимофей брат, Фи...",N,Послание к Филимону святого апостола Павла


In [13]:
ChapsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:3]).text], columns=[('test', 'book', 'chap'), 'text'])
ChapsDf[['test','book','chap']] = pd.DataFrame(list(ChapsDf[('test', 'book', 'chap')]), index=ChapsDf.index, columns=bibleOHCO[:3])
del ChapsDf[('test', 'book', 'chap')]
ChapsDf = ChapsDf.replace({"test": testsDict, "book": booksDict}).sort_values(by=bibleOHCO[:3], ascending=[True, True, True])
ChapsDf = ChapsDf.reset_index().drop(['index'], axis=1)
ChapsDf.index.name = 'c_id'
ChapsDf = ChapsDf.replace({"test":dict(enumerate(BibDf.test.unique())), "book":dict(enumerate(BibDf.book.unique()))})
#ChapsDf = ChapsDf.reset_index().set_index(bibleOHCO[:3])
ChapsDf

,text,test,book,chap
c_id,,,,
0,В начале сотворил Бог небо и землю. Земля же б...,O,Бытие,1
1,Так совершены небо и земля и все воинство их. ...,O,Бытие,2
2,"Змей был хитрее всех зверей полевых, которых с...",O,Бытие,3
3,"Адам познал Еву, жену свою; и она зачала, и ро...",O,Бытие,4
4,Вот родословие Адама: когда Бог сотворил челов...,O,Бытие,5
...,...,...,...,...
1357,"После сего я увидел иного Ангела, сходящего с ...",N,Откровение святого Иоанна Богослова,18
1358,После сего я услышал на небе громкий голос как...,N,Откровение святого Иоанна Богослова,19
1359,"И увидел я Ангела, сходящего с неба, который и...",N,Откровение святого Иоанна Богослова,20


In [28]:
ChapsDf#.reset_index().c_id

,text,test,book,chap
c_id,,,,
0,В начале сотворил Бог небо и землю. Земля же б...,O,Бытие,1
1,Так совершены небо и земля и все воинство их. ...,O,Бытие,2
2,"Змей был хитрее всех зверей полевых, которых с...",O,Бытие,3
3,"Адам познал Еву, жену свою; и она зачала, и ро...",O,Бытие,4
4,Вот родословие Адама: когда Бог сотворил челов...,O,Бытие,5
...,...,...,...,...
1357,"После сего я увидел иного Ангела, сходящего с ...",N,Откровение святого Иоанна Богослова,18
1358,После сего я услышал на небе громкий голос как...,N,Откровение святого Иоанна Богослова,19
1359,"И увидел я Ангела, сходящего с неба, который и...",N,Откровение святого Иоанна Богослова,20


In [29]:
BibDf['c_id'] = ChapsDf.reset_index().c_id

In [30]:
BibDf

,test,book,chap,verse,text,c_id
v_id,,,,,,
0,O,Бытие,1,1,В начале сотворил Бог небо и землю.,0.0
1,O,Бытие,1,2,"Земля же была безвидна и пуста, и тьма над без...",1.0
2,O,Бытие,1,3,И сказал Бог: да будет свет. И стал свет.,2.0
3,O,Бытие,1,4,"И увидел Бог свет, что он хорош, и отделил Бог...",3.0
4,O,Бытие,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве...",4.0
...,...,...,...,...,...,...
37105,N,Откровение святого Иоанна Богослова,22,17,И Дух и невеста говорят: прииди! И слышавший д...,NaN
37106,N,Откровение святого Иоанна Богослова,22,18,И я также свидетельствую всякому слышащему сло...,NaN
37107,N,Откровение святого Иоанна Богослова,22,19,и если кто отнимет что от слов книги пророчест...,NaN


In [31]:
def nat_parse(textDf=BibTextDf, textCol='text', columns=tokenCols): 
    tokenDf = pd.DataFrame(columns=columns)
    for v_id in textDf.index: 
        verseDict = []
        doc = Doc(textDf.loc[v_id][textCol])
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for token in doc.tokens: 
            token.lemmatize(morph_vocab)
        doc.parse_syntax(syntax_parser)
        doc.tag_ner(ner_tagger)
        for sent in enumerate(doc.sents): 
            sent_num = sent[0]
            sent_text = sent[1]
            for token in enumerate([x for x in sent_text.tokens if x.pos!='PUNCT']): 
                token_num = token[0]
                token_text = token[1]
                start = token_text.start
                stop = token_text.stop
                text = token_text.text
                token_id = token_text.id
                head_id = token_text.head_id
                rel = token_text.rel
                pos = token_text.pos
                lemma = token_text.lemma
                # Animacy, Aspect, Case, Degree, Gender, Mood, Number, Person, Tense, VerbForm, Voice
                #print(token_text.feats)
                try: 
                    anim = token_text.feats['Animacy']
                except: 
                    anim = None
                try: 
                    aspect = token_text.feats['Aspect']
                except: 
                    aspect = None
                try: 
                    case = token_text.feats['Case']
                except: 
                    case = None
                try: 
                    degree = token_text.feats['Degree']
                except: 
                    degree = None
                try: 
                    gender = token_text.feats['Gender']
                except: 
                    gender = None
                try: 
                    mood = token_text.feats['Mood']
                except: 
                    mood = None
                try: 
                    number = token_text.feats['Number']
                except: 
                    number = None
                try: 
                    person = token_text.feats['Person']
                except: 
                    person = None
                try: 
                    tense = token_text.feats['Tense']
                except: 
                    tense = None
                try: 
                    verb_form = token_text.feats['VerbForm']
                except: 
                    verb_form = None
                try: 
                    voice = token_text.feats['Voice']
                except: 
                    voice = None
                #print(token)
                tokenDict = {
                    'v_id': v_id,
                    'token_num': token_num, 
                    'start': start, 
                    'stop': stop, 
                    'text': text, 
                    'token_id': token_id, 
                    'head_id': head_id, 
                    'rel': rel, 
                    'pos': pos, 
                    'lemma': lemma, 
                    'anim': anim, 
                    'aspect': aspect, 
                    'case': case, 
                    'degree': degree, 
                    'gender': gender, 
                    'mood': mood, 
                    'number': number, 
                    'person': person, 
                    'tense': tense, 
                    'verb_form': verb_form, 
                    'voice': voice
                }
                verseDict.append(tokenDict)
            #print(sent)
            verseDf = pd.DataFrame(verseDict, columns=columns)
        tokenDf = pd.concat([tokenDf, verseDf])
    return tokenDf

In [38]:
%%time
nat_parse(ChapsDf.iloc[:10])#.set_index(['v_id', 'token_num'])

CPU times: user 10.6 s, sys: 10.1 s, total: 20.7 s
Wall time: 2.9 s


,v_id,token_num,start,stop,text,token_id,head_id,rel,pos,lemma,...,aspect,case,degree,gender,mood,number,person,tense,verb_form,voice
0,0,0,0,1,В,1_1,1_2,case,ADP,в,...,None,None,None,None,None,None,None,None,None,None
1,0,1,2,8,начале,1_2,1_3,obl,NOUN,начало,...,None,Loc,None,Neut,None,Sing,None,None,None,None
2,0,2,9,17,сотворил,1_3,1_0,root,VERB,сотворить,...,Perf,None,None,Masc,Ind,Sing,None,Past,Fin,Act
3,0,3,18,21,Бог,1_4,1_3,nsubj,PROPN,бог,...,None,Nom,None,Masc,None,Sing,None,None,None,None
4,0,4,22,26,небо,1_5,1_3,obj,NOUN,небо,...,None,Acc,None,Neut,None,Sing,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,9,3,1976,1982,народы,27_4,27_3,nsubj,NOUN,народ,...,None,Nom,None,Masc,None,Plur,None,None,None,None
318,9,4,1983,1985,на,27_5,27_6,case,ADP,на,...,None,None,None,None,None,None,None,None,None,None
319,9,5,1986,1991,земле,27_6,27_4,nmod,NOUN,земля,...,None,Loc,None,Fem,None,Sing,None,None,None,None
320,9,6,1992,1997,после,27_7,27_8,case,ADP,после,...,None,None,None,None,None,None,None,None,None,None


In [40]:
%%time
nat_parse(BibTextDf.iloc[:321])#.set_index(['v_id', 'token_num'])

CPU times: user 22.8 s, sys: 22.7 s, total: 45.5 s
Wall time: 6.1 s


,v_id,token_num,start,stop,text,token_id,head_id,rel,pos,lemma,...,aspect,case,degree,gender,mood,number,person,tense,verb_form,voice
0,0,0,0,1,В,1_1,1_2,case,ADP,в,...,None,None,None,None,None,None,None,None,None,None
1,0,1,2,8,начале,1_2,1_3,obl,NOUN,начало,...,None,Loc,None,Neut,None,Sing,None,None,None,None
2,0,2,9,17,сотворил,1_3,1_0,root,VERB,сотворить,...,Perf,None,None,Masc,Ind,Sing,None,Past,Fin,Act
3,0,3,18,21,Бог,1_4,1_3,nsubj,PROPN,бог,...,None,Nom,None,Masc,None,Sing,None,None,None,None
4,0,4,22,26,небо,1_5,1_3,obj,NOUN,небо,...,None,Acc,None,Neut,None,Sing,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,320,5,24,30,скотом,1_6,1_5,iobj,NOUN,скот,...,None,Ins,None,Masc,None,Sing,None,None,None,None
6,320,6,32,33,и,1_8,1_9,cc,CCONJ,и,...,None,None,None,None,None,None,None,None,None,None
7,320,7,34,42,серебром,1_9,1_6,conj,NOUN,серебро,...,None,Ins,None,Neut,None,Sing,None,None,None,None
8,320,8,44,45,и,1_11,1_12,cc,CCONJ,и,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
tokenDf.to_pickle('./proc/tokenDf.pkl')